In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import cv2
import os 
import random

2022-07-06 21:05:44.369756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 21:05:44.369773: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
DIRECTORY = r"/home/user/ML/Deep Learning/CNN/Covid/train"
TEST = r"/home/user/ML/Deep Learning/CNN/Covid/test"

data = []

CATEGORIES = ["Covid","Normal","Viral Pneumonia"]

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY,category)
    
    label = CATEGORIES.index(category)
    
    for img in os.listdir(folder):
        img = os.path.join(folder,img)
        img_arr = cv2.imread(img)
        img_arr = cv2.resize(img_arr,(100,100))
#         plt.imshow(img_arr)
#         plt.show()
        data.append([img_arr,label])


In [8]:
# Pre Processing 

IMG_SIZE = 224
BATCH_SIZE = 64

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    validation_split= 0.1
)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
    validation_split= 0.1
)

train_datagen = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size= BATCH_SIZE,
    subset="training"
)

test_datagen = test_datagen.flow_from_directory(
    TEST,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size= BATCH_SIZE,
    subset="validation"
)

Found 226 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [9]:
train_datagen.class_indices

{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [18]:
# Model Generate 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

model = Sequential()

model.add(Conv2D(filters=32,padding="same",strides=2,kernel_size=3,activation="relu",input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPooling2D(pool_size=2,strides=2))

model.add(Flatten())
model.add(Dense(3,activation="softmax"))

In [20]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
history = model.fit(train_datagen,epochs=10,validation_data=test_datagen)

Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 2.4656 - accuracy: 0.4159 - val_loss: 1.7789 - val_accuracy: 0.3333
Epoch 2/10
4/4 [==============================] - 10s 3s/step - loss: 1.2868 - accuracy: 0.5354 - val_loss: 1.1092 - val_accuracy: 0.6667
Epoch 3/10
4/4 [==============================] - 10s 3s/step - loss: 1.0126 - accuracy: 0.6195 - val_loss: 1.0175 - val_accuracy: 0.6667
Epoch 4/10
4/4 [==============================] - 10s 2s/step - loss: 0.6060 - accuracy: 0.7478 - val_loss: 0.6419 - val_accuracy: 0.6667
Epoch 5/10
4/4 [==============================] - 10s 2s/step - loss: 0.5869 - accuracy: 0.7920 - val_loss: 0.5709 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 10s 2s/step - loss: 0.5272 - accuracy: 0.7920 - val_loss: 0.6574 - val_accuracy: 0.6667
Epoch 7/10
4/4 [==============================] - 10s 2s/step - loss: 0.4205 - accuracy: 0.8761 - val_loss: 0.5381 - val_accuracy: 0.6667
Epoch 8/10
4/4 [==================

In [21]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 3)                 301059    
                                                                 
Total params: 301,955
Trainable params: 301,955
Non-trainable params: 0
_________________________________________________________________


In [23]:
np.array(history.history['accuracy']).mean()

0.7292035460472107

In [34]:
from keras.preprocessing import image
path = "/home/user/ML/Deep Learning/CNN/Covid/net/"
img = path + "in.jpeg"

img_pred = tf.keras.utils.load_img(img,target_size=(IMG_SIZE,IMG_SIZE))

img_pred = tf.keras.utils.img_to_array(img_pred)
img_pred = img_pred / 255
img_pred = np.expand_dims(img_pred,axis=0)
result = model.predict(img_pred)

print(CATEGORIES[np.argmax(result)])

1/1 [==============================] - 0s 16ms/step
Covid
